# type--类工厂

有时，我觉得应该有类似`nametuple`的工厂函数，用于创建可变对象.假设我在编写一个宠物店应用程序，我想把狗的数据当作简单的记录处理.编写下面的样板代码让人厌烦：

In [1]:
class Dog:
    def __init__(self, name, weight, owner):
        self.name = name
        self.weight = weight
        self.owner = owner

In [2]:
rex = Dog('Rex', 30, 'Bob')
rex

各个字段名称出现了三次。写了这么多样板代码，甚至字符串表示形式都不友好.

参考`namedtuple`,下面我们创建一个`record_factory`函数，即时创建简单的类

In [4]:
def record_factory(cls_name, field_names):
    try:
        field_names = field_names.replace(',', ' ').split()
    except AttributeError: # 不能调用.replace或.split方法
        pass # 假定field_names本就是标识符组成的序列
    field_names = tuple(field_names) 
    
    def __init__(self, *args, **kwargs): 
        attrs = dict(zip(self.__slots__, args))
        attrs.update(kwargs)
        for name, value in attrs.items():
            setattr(self, name, value)
    
    def __iter__(self): 
        for name in self.__slots__:
            yield getattr(self, name)
    def __repr__(self): 
        values = ', '.join('{}={!r}'.format(*i) for i in zip(self.__slots__, self))
        return '{}({})'.format(self.__class__.__name__, values)
    
    cls_attrs = dict(__slots__ = field_names, __init__ = __init__,__iter__ = __iter__,__repr__ = __repr__)
    return type(cls_name, (object,), cls_attrs)

In [14]:
Cat = record_factory('Cat', 'name weight owner') 

In [15]:
rex = Cat('Rex', 30, 'Bob')
rex

Cat(name='Rex', weight=30, owner='Bob')

In [16]:
name, weight, _ = rex

In [17]:
name, weight

('Rex', 30)

In [18]:
"{2}'s dog weighs {1}kg".format(*rex)

"Bob's dog weighs 30kg"

In [19]:
rex.weight = 32
rex

Cat(name='Rex', weight=32, owner='Bob')

In [20]:
Cat.__mro__

(__main__.Cat, object)

可以看出上面的工厂函数核心就在于`type()`的使用.通常，我们把`type`视作函数，因为我们像函数那样使用它，例如，调用`type(my_object)`
获取对象所属的类——作用与`my_object.__class__`相同.

然而，`type`是一个类。当成类使用时，传入三个参数可以新建一个类：

```python
MyClass = type('MyClass', (MySuperClass, MyMixin),{'x': 42, 'x2': lambda self: self.x * 2})
```
type的三个参数分别是`name`、`bases` 和`dict`.最后一个参数是一个映射，指定新类的属性名和值.

# 元类

元类是制造类的工厂，不过不是函数而是类.

根据Python对象模型，类是对象，因此类肯定是另外某个类的实例.默认情况下，Python
中的类是type类的实例.也就是说，type 是大多数内置的类和用户定义的类的元类

In [21]:
'spam'.__class__

str

In [22]:
str.__class__

type

In [23]:
type.__class__

type

为了避免无限回溯，type 是其自身的实例，如最后一行所示。
注意，我没有说str 或其他对象继承自type.我的意思是，str 和其他对象是type 的实例。这两个类是object的子类。下图是他们的关系
![](source/type.PNG)
两个示意图都是正确的。左边的示意图强调`str`、`type` 和`LineItem` 是`object` 的子类。右
边的示意图则清楚地表明`str`、`object` 和`LineItem` 是`type` 的实例，因为它们都是类.

除了type，标准库中还有一些别的元类，例如`ABCMeta` 和`Enum`。如下述代码片段所示，`collections.Iterable` 所属的类是`abc.ABCMeta`.`Iterable`是抽象类，而`ABCMeta`不是——
不管怎样，`Iterable` 是`ABCMeta`的实例

In [24]:
import collections

In [25]:
collections.Iterable.__class__

abc.ABCMeta

In [26]:
import abc

In [27]:
abc.ABCMeta.__class__

type

In [28]:
abc.ABCMeta.__mro__

(abc.ABCMeta, type, object)

向上追溯，`ABCMeta`最终所属的类也是`type`。所有类都直接或间接地是`type` 的实例，不过只有元类同时也是`type` 的子类。若想理解元类，一定要知道这种关系：元类（如`ABCMeta`）从`type` 类继承了构建类的能力.
![](source/abc.PNG)

我们要抓住的重点是，所有类都是type 的实例，但是元类还是type 的子类，因此可以作
为制造类的工厂。具体来说，元类可以通过实现__init__ 方法定制实例。元类的__init__
方法可以做到类装饰器能做的任何事情，但是作用更大.

In [31]:
%%writefile evaltime_meta.py
from evalsupport import deco_alpha
from evalsupport import MetaAleph
print('<[1]> evaltime_meta module start')

@deco_alpha
class ClassThree():
    print('<[2]> ClassThree body')
    def method_y(self):
        print('<[3]> ClassThree.method_y')
        
class ClassFour(ClassThree):
    print('<[4]> ClassFour body')
    def method_y(self):
        print('<[5]> ClassFour.method_y')
class ClassFive(metaclass=MetaAleph):
    print('<[6]> ClassFive body')
    def __init__(self):
        print('<[7]> ClassFive.__init__')
    def method_z(self):
        print('<[8]> ClassFive.method_z')
class ClassSix(ClassFive):
    print('<[9]> ClassSix body')
    def method_z(self):
        print('<[10]> ClassSix.method_z')
        
if __name__ == '__main__':
    print('<[11]> ClassThree tests', 30 * '.')
    three = ClassThree()
    three.method_y()
    print('<[12]> ClassFour tests', 30 * '.')
    four = ClassFour()
    four.method_y()
    print('<[13]> ClassFive tests', 30 * '.')
    five = ClassFive()
    five.method_z()
    print('<[14]> ClassSix tests', 30 * '.')
    six = ClassSix()
    six.method_z()
print('<[15]> evaltime_meta module end')

Overwriting evaltime_meta.py


In [33]:
import evaltime_meta

<[100]> evalsupport module start
<[400]> MetaAleph body
<[700]> evalsupport module end
<[1]> evaltime_meta module start
<[2]> ClassThree body
<[200]> deco_alpha
<[4]> ClassFour body
<[6]> ClassFive body
<[500]> MetaAleph.__init__
<[9]> ClassSix body
<[500]> MetaAleph.__init__
<[15]> evaltime_meta module end


In [32]:
!python evaltime_meta.py

<[100]> evalsupport module start
<[400]> MetaAleph body
<[700]> evalsupport module end
<[1]> evaltime_meta module start
<[2]> ClassThree body
<[200]> deco_alpha
<[4]> ClassFour body
<[6]> ClassFive body
<[500]> MetaAleph.__init__
<[9]> ClassSix body
<[500]> MetaAleph.__init__
<[11]> ClassThree tests ..............................
<[300]> deco_alpha:inner_1
<[12]> ClassFour tests ..............................
<[5]> ClassFour.method_y
<[13]> ClassFive tests ..............................
<[7]> ClassFive.__init__
<[600]> MetaAleph.__init__:inner_2
<[14]> ClassSix tests ..............................
<[7]> ClassFive.__init__
<[600]> MetaAleph.__init__:inner_2
<[15]> evaltime_meta module end
